In [1]:
import sys
from pathlib import Path
import pandas as pd

# Determine the root directory (adjust the number of parents as needed)
root_dir = Path.cwd().parents[0]  # Adjust based on your folder structure

# Add the root directory to sys.path
sys.path.append(str(root_dir))
print(sys.path)

from models.naive_bayes import NaiveBayes




['c:\\Users\\cocol\\Documents\\Cours\\3A\\PIC\\defensive\\notebooks', 'c:\\ProgramData\\Anaconda3\\python311.zip', 'c:\\ProgramData\\Anaconda3\\DLLs', 'c:\\ProgramData\\Anaconda3\\Lib', 'c:\\ProgramData\\Anaconda3', '', 'C:\\Users\\cocol\\AppData\\Roaming\\Python\\Python311\\site-packages', 'c:\\ProgramData\\Anaconda3\\Lib\\site-packages', 'c:\\ProgramData\\Anaconda3\\Lib\\site-packages\\win32', 'c:\\ProgramData\\Anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\ProgramData\\Anaconda3\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\cocol\\Documents\\Cours\\3A\\PIC\\defensive']


In [2]:
from datasets import load_dataset

dataset = load_dataset("TrainingDataPro/email-spam-classification")


In [3]:
dataset_df = pd.DataFrame(dataset['train'])
dataset_df.rename(columns={'title': 'subject', 'text': 'body'}, inplace=True)

dataset_df['ground_truth'] = dataset_df['type'].apply(lambda x: 1 if x == 'spam' else 0)

dataset_df.drop(columns=['type'], inplace=True)
dataset_df

,subject,body,ground_truth
0,?? the secrets to SUCCESS,"Hi James,\n\nHave you claim your complimentary...",1
1,?? You Earned 500 GCLoot Points,"\nalt_text\nCongratulations, you just earned\n...",0
2,?? Your GitHub launch code,"Here's your GitHub launch code, @Mortyj420!\n ...",0
3,[The Virtual Reward Center] Re: ** Clarifications,"Hello,\n \nThank you for contacting the Virtua...",0
4,"10-1 MLB Expert Inside, Plus Everything You Ne...","Hey Prachanda Rawal,\n\nToday's newsletter is ...",1
...,...,...,...
79,Your application for the position of Child Pr...,"Dear Maryam, \n\n \n\nI would like to thank yo...",0
80,Your Kilimall Account is Ready - Shopping Now!,"Dear Customer,\n\nWelcome to Kilimall, Thanks ...",0
81,Your Steam account: Access from new web or mob...,"Dear vladis163rus,\nHere is the Steam Guard co...",0
82,Your uploaded document is rejected,View In Browser | Log in\n \n \n\nSkrill logo\...,0


In [4]:
classifier = NaiveBayes(dataset_df)

classifier.train(dataset_df)

Accuracy: 0.7647058823529411 0.7647058823529411


0.7647058823529411